# Ejemplo de proceso Data Science #2: Deteccion de contenido adulto en paginas web.


En este ejemplo creamos una aplicacion, que dada una url, nos dirá si dicha url es de contenido para adultos o no.


Usaremos Scrapy para scrapear el dataset de entrenamiento, NTLK para el procesado de lenguaje natural y Scikit-Learn para el Machine Learning

## Definición del Problema


Una empresa de ropa de bebés ha gastado una cantidad no considerable en anuncios online. Sin embargo, ha recibido noticias de que sus anuncios se han mostrado en páginas de contenido adulto. El CEO quiere analizar los datos históricos de anuncios mostrados y asegurarse de que no se han mostrado sus anuncios en páginas adultas 

# 1. Obtención del dataset de entrenamiento.

Crearemos una [araña](https://github.com/manugarri/jornada_data_science_info_2017/blob/master/ejemplos/Ejemplo2/spider.py) a la que le daremos un listado de links de agregadores de contenido adulto por una parte, y otro listado de links de contenido no adulto. Dicha araña simplemente parseara todo el HTML de cada link en dichas páginas.

# Leemos el dataset con los contenidos adultos y no adultos

In [1]:
import json
import pandas as pd

with open("data.jsonlines") as fname:
    lines = fname.readlines()
    rows = [json.loads(l) for l in lines]
    data = pd.DataFrame(rows)

In [2]:
data.shape

(853, 3)

# El dataset inicial contiene la url, el contenido html y una etiqueta que es 1 si el contenido es adulto y un 0 si no lo es

In [3]:
data.head(10)

,label,text,url
0,1,User:\n Pass:\n \n \n\n \n ...,http://www.kuntfutube.com/videos/all/date-0/
1,1,Home | \n Categ...,http://www.pk5.net/
2,1,youpunish.com\tEnlaces relacionadosEste domini...,http://ww1.youpunish.com
3,1,FuckingMonth Filter by post typeAll postsTextP...,http://g-o-z-a-r.tumblr.com/archive/
4,1,"(function() { document.write('<iframe alt="""" a...",http://www.yobt.tv/
5,1,SexgifMonth Filter by post typeAll postsTextPh...,http://sexgif-net.tumblr.com/archive/
6,1,phantomPopunder = (function () {\r\n\tvar phan...,http://vintage-erotica-forum.com/
7,1,Sign-In or Join\r\n\t\t \r\n\t \...,http://www.icanhazchat.com/welcome?gonewild
8,1,sonnbXenGalleryEnableInterlace = 0;\n\n\n\n ...,http://forum.xnxx.com/
9,1,Forumophilia\n\n\n\n\n\nRegister \n\nLog in ...,http://www.forumophilia.com


In [4]:
data.tail(10)

,label,text,url
843,0,Discover and buy amazing products.\nGet The Ap...,https://keep.com/
844,0,GENIUS\n \n \n\n \n \n ...,http://genius.com/
845,0,"if ((navigator.doNotTrack===""yes"") || (navigat...",http://porhomme.com/
846,0,YAHOO.util.Event.onDOMReady(function() {\n ...,http://www.freemanbrand.com/
847,0,(function(){\r\n var bsa = document.createEle...,http://everydaycarry.com/
848,0,body { display: none !important; }\n\n if (s...,https://www.twitch.tv/
849,0,SHOP\n \n \n...,https://grandst.com/
850,0,Skip navigation\n \n \n \n \nESUploadSign ...,https://www.youtube.com/user/DailyTekk
851,0,"window._sharedData = {""country_code"":""ES"",""lan...",https://www.instagram.com/dailytekk/
852,0,"Cookies help us provide, protect and improve F...",https://www.facebook.com/login.php?skip_api_lo...


# Extraemos informacion de las tags HTML relevantes

In [5]:
from parsel import Selector

def parse_url(url_text):
    sel = Selector(text=url_text, type="html")
    meta_tags = ['keywords', 'description', 'category', 'RATING']
    tags = []
    for tag in meta_tags:
        tags.append( sel.xpath('//meta[@name="{}"]/@content'.format(tag)).extract_first())
    text = ''.join(sel.xpath("//body//text()").extract()).strip()
    text += ' '.join([i for i in tags if i])
    return text

In [6]:
data['processed_text'] = data["text"].apply(parse_url)

In [7]:
data.head()

,label,text,url,processed_text
0,1,User:\n Pass:\n \n \n\n \n ...,http://www.kuntfutube.com/videos/all/date-0/,User:\n Pass:\n \n \n\n \n ...
1,1,Home | \n Categ...,http://www.pk5.net/,Home | \n Categ...
2,1,youpunish.com\tEnlaces relacionadosEste domini...,http://ww1.youpunish.com,youpunish.com\tEnlaces relacionadosEste domini...
3,1,FuckingMonth Filter by post typeAll postsTextP...,http://g-o-z-a-r.tumblr.com/archive/,FuckingMonth Filter by post typeAll postsTextP...
4,1,"(function() { document.write('<iframe alt="""" a...",http://www.yobt.tv/,(function() { document.write('');})();\n \t...


# Usamos Procesado de Lenguaje Natural para limpiar el texto html de las paginas web y extraer las palabras relevantes

In [8]:
import string
import nltk
from nltk import stem, word_tokenize
import string
import numpy
from nltk.corpus import stopwords

nltk.download('punkt')

stemmer = stem.PorterStemmer()
stemmer = stem.SnowballStemmer("english")

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

[nltk_data] Downloading package punkt to /home/manuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Creamos un CountVectorizer, que convierte una lista de palabras en una matriz de las palabras mas comunes y el numero de veces que aparecen en cada linea

In [77]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(strip_accents='unicode',ngram_range=(1,3), stop_words='english', tokenizer=tokenize)

# El output de vectorizer tiene este aspecto

In [78]:
vectorizer.fit(data.processed_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        strip_accents='unicode', token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7ff9b2cfb2f0>, vocabulary=None)

In [79]:
text_vector = vectorizer.transform(data.processed_text)

In [80]:
text_vector.todense()

matrix([[15,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ..., 
        [ 3,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]])

In [81]:
text_vector.shape

(853, 818295)

In [82]:
list(vectorizer.vocabulary_.keys())[:10]

['user',
 'pass',
 'forgot',
 'regist',
 'kuntfu',
 '1st',
 'asian',
 'tube',
 'site',
 'main']

# Creamos un pipeline (basicamente una lista de pasos que aplicar al dataset) con el countVectorizer y el Algoritmo SVC

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
pipeline = Pipeline([("vectorizer", vectorizer), ("svm", SVC(probability=True, class_weight='balanced'))])

# Entrenamos el modelo

In [84]:
pipeline_fit = pipeline.fit(data.processed_text, data.label)

# Creamos funcion para detectar páginas adultas

In [85]:
from urllib.parse import urljoin
import requests
from urllib.parse import urlparse

def get_url_text(url):
    request = requests.get(url)
    return request.text

def detect_adult_page(url):
    url_text = get_url_text(url)
    data = parse_url(url_text)
    probs =  pipeline_fit.predict_proba([data])[0]
    print("Probabilities for url {}: Not Adult:{}; Adult:{}".format(url, probs[0], probs[1]))

# Testeamos

In [89]:
print("Paginas de contenido Adulto")
detect_adult_page("http://www.redtube.com")
detect_adult_page("http://www.xvideos.com")

print("\n\nPaginas de contenido No Adulto")
detect_adult_page("http://www.nytimes.com/")
detect_adult_page("http://medium.com/@ericfeldman93/my-nephew-vs-ml-9e4519af499a")

Paginas de contenido Adulto
Probabilities for url http://www.redtube.com: Not Adult:1.196104442551119e-10; Adult:0.9999999998803895
Probabilities for url http://www.xvideos.com: Not Adult:0.18846761985799812; Adult:0.8115323801420019


Paginas de contenido No Adulto
Probabilities for url http://www.nytimes.com/: Not Adult:0.99994495734706; Adult:5.5042652939848096e-05
Probabilities for url http://medium.com/@ericfeldman93/my-nephew-vs-ml-9e4519af499a: Not Adult:0.6070758294690424; Adult:0.3929241705309576
